# cadCAD Tutorials: The Robot and the Marbles Part 5 - Networks

To expand upon our previous examples, we will introduce the concept of using a graph network object that is updated during each state update. The ability to essential embed a graph 'database' into a state is a game changer for scalability, allowing increased complexity with multiple agents or components is represented, easily updated. Below, building upon our previous examples, we will represent the Robots and Marbles example with n boxes, and a variable number of marbles. 

## Behavior and Mechanisms:
* A network of robotic arms is capable of taking a marble from their one of their boxes and dropping it into the other one. 
* Each robotic arm in the network only controls two boxes and they act by moving a marble from one box to the other.
* Each robotic arm is programmed to take one marble at a time from the box containing the most significant number of marbles and drop it in the other box. It repeats that process until the boxes contain an equal number of marbles.
* For our analysis of this system, suppose we are only interested in monitoring the number of marbles in only their two boxes.

In [1]:
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD.configuration import Configuration
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
seed = 12302
#from copy import deepcopy
np.random.seed(seed)

%matplotlib inline

# define global variables
T = 25 #iterations in our simulation
boxes=5 #number of boxes in our network
m= 2 #for barabasi graph type number of edges is (n-2)*m

We create a [Barabási–Albert](https://en.wikipedia.org/wiki/Barab%C3%A1si%E2%80%93Albert_model) graph and then fill the 5 boxes with between 1 and 10 balls. You can create as many different nodes or types of nodes as needed

In [2]:
# create graph object with the number of boxes as nodes
network = nx.barabasi_albert_graph(boxes, m, seed=seed)

# add balls to box nodes
for node in network.nodes:
    network.nodes[node]['balls'] = np.random.randint(1,10)

In [3]:
# we initialize the cadCAD state as a network object
# we add state variables for metrics of the graph we're interested in
initial_conditions = {'network':network, 'size':network.number_of_nodes(), 'center':nx.center(network)}

In [4]:
#Behavior: node by edge dimensional operator
#input the states of the boxes output the deltas along the edges

# We specify the robotic networks logic in a Policy/Behavior Function
# unlike previous examples our policy controls a vector valued action, defined over the edges of our network
def robotic_network(params, step, sL, s):
    network = s['network']
    delta_balls = {}
    for e in network.edges:
        src = e[0]
        dst = e[1]
        #transfer one ball across the edge in the direction of more balls to less
        delta_balls[e] = np.sign(network.nodes[src]['balls']-network.nodes[dst]['balls'])
    return({'delta': delta_balls})

In [5]:
def grow_network(params, step, sL, s, _input):
    network = s['network'] #deepcopy(s['network'])
    new_node = network.number_of_nodes()
    network.add_edge(new_node, np.random.randint(0,new_node))
    network.nodes[new_node]['balls'] = np.random.randint(1,10)
    return ('network', network)

#mechanism: edge by node dimensional operator
#input the deltas along the edges and update the boxes

# We make the state update functions less "intelligent",
# ie. they simply add the number of marbles specified in _input 
# (which, per the policy function definition, may be negative)

def update_network(params, step, sL, s, _input):
    network = s['network'] #deepcopy(s['network']) 
    delta_balls = _input['delta']
    for e in network.edges:
        move_ball = delta_balls[e]
        src = e[0]
        dst = e[1]
        if (network.nodes[src]['balls'] >= move_ball) and (network.nodes[dst]['balls'] >= -move_ball):
            network.nodes[src]['balls'] = network.nodes[src]['balls']-move_ball
            network.nodes[dst]['balls'] = network.nodes[dst]['balls']+move_ball
                        
    return ('network', network)

def calculate_size(params, step, sL, s, _input):
    return('size', s['network'].number_of_nodes())

def calculate_center(params, step, sL, s, _input):
    return('center', nx.center(s['network']))

In [6]:
# wire up the mechanisms and states
partial_state_update_blocks = [
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
        },
        'variables': { # The following state variables will be updated simultaneously
            'network': grow_network
        }
    },
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'action': robotic_network
        },
        'variables': { # The following state variables will be updated simultaneously
            'network': update_network
        }
    },
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
        },
        'variables': { # The following state variables will be updated simultaneously
            'size': calculate_size,
            'center': calculate_center
        }
    }
]

In [7]:
params= {
    'network': [network]
    }
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
time_periods_per_run = 100
monte_carlo_runs = 1

from cadCAD.configuration.utils import config_sim
simulation_parameters = config_sim({
    'T': range(time_periods_per_run),
    'N': monte_carlo_runs,
    'M': params
})

In [8]:
from cadCAD.configuration import append_configs
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# The configurations above are then packaged into a `Configuration` object
append_configs(
    initial_state=initial_conditions, #dict containing variable names and initial values
    partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
    sim_configs=simulation_parameters #dict containing simulation parameters
)
from tabulate import tabulate
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD import configs
import pandas as pd

exec_mode = ExecutionMode()
multi_proc_ctx = ExecutionContext(context=exec_mode.multi_proc)
run = Executor(exec_context=multi_proc_ctx, configs=configs)

In [9]:
i = 0
verbose = False
results = {}
for raw_result, tensor_field in run.execute():
    result = pd.DataFrame(raw_result)
    if verbose:
        print()
        print(f"Tensor Field: {type(tensor_field)}")
        print(tabulate(tensor_field, headers='keys', tablefmt='psql'))
        print(f"Output: {type(result)}")
        print(tabulate(result, headers='keys', tablefmt='psql'))
        print()
    results[i] = {}
    results[i]['result'] = result
    results[i]['simulation_parameters'] = simulation_parameters[i]
    i += 1

multi_proc: [<cadCAD.configuration.Configuration object at 0x105726908>]


In [10]:
results[0]['result']

,center,network,run,size,substep,timestep
0,"[0, 1, 2, 3, 4]","(0, 1, 2, 3, 4)",1,5,0,0
1,"[0, 1, 2, 3, 4]","(0, 1, 2, 3, 4, 5)",1,5,1,1
2,"[0, 1, 2, 3, 4]","(0, 1, 2, 3, 4, 5)",1,5,2,1
3,"[0, 2, 3]","(0, 1, 2, 3, 4, 5)",1,6,3,1
4,"[0, 2, 3]","(0, 1, 2, 3, 4, 5, 6)",1,6,1,2
5,"[0, 2, 3]","(0, 1, 2, 3, 4, 5, 6)",1,6,2,2
6,[3],"(0, 1, 2, 3, 4, 5, 6)",1,7,3,2
7,[3],"(0, 1, 2, 3, 4, 5, 6, 7)",1,7,1,3
8,[3],"(0, 1, 2, 3, 4, 5, 6, 7)",1,7,2,3
9,[3],"(0, 1, 2, 3, 4, 5, 6, 7)",1,8,3,3
